<a href="https://colab.research.google.com/github/InShanu/Projects/blob/main/insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import pandas as pd

# Load the CSV file
file_path = '/content/Insights.csv'
data = pd.read_csv(file_path)

# Expand the 'Tags' column to create one row per tag
data_expanded = data.assign(Tags=data['Tags'].str.strip('[]').str.split(',')).explode('Tags')
data_expanded['Tags'] = data_expanded['Tags'].str.strip(' "')

# Ensure tags and programs are distinct
data_expanded = data_expanded.drop_duplicates(subset=['Tags', 'Program'])

# Prepare the data for the matrix chart
matrix_data = data_expanded.pivot_table(index='Tags', columns='Program', aggfunc='size', fill_value=0)
tags = matrix_data.index.tolist()
programs = matrix_data.columns.tolist()
matrixData = []
for i, tag in enumerate(tags):
    for j, program in enumerate(programs):
        matrixData.append([j, i, matrix_data.at[tag, program]])



# Prepare the data for the tree map
treemap_data = data.groupby(['Domain', 'Program']).size().reset_index(name='counts')
treemapData = []
for domain in treemap_data['Domain'].unique():
    children = treemap_data[treemap_data['Domain'] == domain].apply(lambda x: {"name": x['Program'], "value": x['counts']}, axis=1).tolist()
    treemapData.append({"name": domain, "value": sum([child['value'] for child in children]), "children": children})
# Prepare data for bar and pie charts
insights_over_time = data['Date Identified'].value_counts().sort_index()
dates = insights_over_time.index.tolist()
counts = insights_over_time.values.tolist()

domain_distribution = data['Domain'].value_counts()
domains = domain_distribution.index.tolist()
domain_counts = domain_distribution.values.tolist()
pieData = [{"value": count, "name": domain} for count, domain in zip(domain_counts, domains)]


In [67]:
from IPython.display import display, HTML

html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Insights Data Analysis 7/8/24</title>
    <script src="https://cdn.jsdelivr.net/npm/echarts/dist/echarts.min.js"></script>
</head>
<body>
    <h1>Insights Data Analysis 7/8/24</h1>

    <div id="barChart" style="width: 1000px; height: 600px;"></div>
    <div id="pieChart" style="width: 1000px; height: 600px;"></div>
    <div id="matrixChart" style="width: 1000px; height: 600px;"></div>
    <div id="treeMap" style="width: 1000px; height: 600px;"></div>

    <script>
    var barChartDom = document.getElementById('barChart');
    var barChart = echarts.init(barChartDom);
    var barOption = {{
        xAxis: {{
            type: 'category',
            data: {dates}
        }},
        yAxis: {{
            type: 'value'
        }},
        series: [{{
            data: {counts},
            type: 'bar'
        }}]
    }};
    barChart.setOption(barOption);

    var pieChartDom = document.getElementById('pieChart');
    var pieChart = echarts.init(pieChartDom);
    var pieOption = {{
        series: [{{
            type: 'pie',
            data: {pieData}
        }}]
    }};
    pieChart.setOption(pieOption);

    var matrixChartDom = document.getElementById('matrixChart');
    var matrixChart = echarts.init(matrixChartDom);
    var matrixOption = {{
        tooltip: {{
            position: 'top'
        }},
        grid: {{
            height: '50%',
            top: '10%'
        }},
        xAxis: {{
            type: 'category',
            data: {programs},
            axisLabel: {{ rotate: 45, interval: 0 }}
        }},
        yAxis: {{
            type: 'category',
            data: {tags}
        }},
        visualMap: {{
            min: 0,
            max: {max([d[2] for d in matrixData])},
            calculable: true,
            orient: 'horizontal',
            left: 'center',
            bottom: '15%'
        }},
        series: [{{
            name: 'Frequency',
            type: 'heatmap',
            data: {matrixData},
            label: {{ show: true }},
            emphasis: {{
                itemStyle: {{
                    shadowBlur: 10,
                    shadowColor: 'rgba(0, 0, 0, 0.5)'
                }}
            }}
        }}]
    }};
    matrixChart.setOption(matrixOption);

    var treeMapDom = document.getElementById('treeMap');
    var treeMap = echarts.init(treeMapDom);
    var treeMapOption = {{
        series: [{{
            type: 'treemap',
            data: {treemapData},
            label: {{
                show: true,
                formatter: '{{b}}: {{c}}'
            }}
        }}]
    }};
    treeMap.setOption(treeMapOption);
    </script>
</body>
</html>
"""

# Display the HTML content
display(HTML(html_content))


In [69]:
with open('insights_dashboard.html', 'w') as f:
    f.write(html_content)


In [70]:
from google.colab import files
files.download('insights_dashboard.html')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>